In [6]:
import requests
import re
from bs4  import BeautifulSoup

# 쿠팡 노트북 검색 결과를 페이지를 지정해서 출력
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36"}

# 페이지 선택
def coupang(page):
    # 페이지 정보 출력
    print("현재 페이지:" + str(page))
    # 페이지에 따라 url 변경
    url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=\
    scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page={0}&\
    rocketAll=false&searchIndexingToken=&backgroundColor=".format(page)

    res = requests.get(url, headers=headers)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, "lxml")

    items = soup.find_all("li", attrs={"class":re.compile("^search-product")})

    for i in items:
        # 로켓배송만 포함
        rocket = i.find("span", attrs={"class":"badge rocket"})
        if not rocket:
            print("로켓 배송만 포함합니다")
            continue
            
        # 삼성전자 상품 제외
        name = i.find("div", attrs={"class":"name"}).get_text() # 제품명
        if "삼성전자" in name:
            print("삼성전자 제품 제외")
            continue

        price = i.find("strong", attrs={"class":"price-value"}).get_text() # 가격

        # 리뷰 50개 이상, 평점 4.2 이상만 조회
        rate = i.find("em", attrs={"class":"rating"}) # 평점 - 평점이 없는 경우가 있었음
        if rate:
            rate = rate.get_text()
        else:
            print("평점 없음")
            continue

        rate_cnt = i.find("span", attrs={"class":"rating-total-count"}) # 평점 수 (30)
        if rate_cnt:
            rate_cnt = rate_cnt.get_text()[1:-1] # 평점 수 옆의 괄호 제거
        else:
            print("평점 수 없음")
            continue
        
        # 링크 추가
        link = i.find("a", attrs={"class":"search-product-link"})["href"]
        
        # 최중 출력
        if float(rate) >= 4.2 and int(rate_cnt) >= 50:
            print(f"제품명: {name}")
            print(f"가격: {price}원")
            print(f"평점: {rate} ({rate_cnt})개")
            print(f"링크: {'https://www.coupang.com' + link}")
            print("-----------------------------------------------------------------------------------------------------------------")

coupang(2)

현재 페이지:2
제품명: Apple 2020년 맥북 프로 13 (M1 칩셋 8코어 CPU 8코어 GPU), 8GB, SSD 256GB, 스페이스 그레이
가격: 1,586,800원
평점: 5.0 (575)개
링크: https://www.coupang.com/vp/products/4322481029?itemId=5033260815&vendorItemId=72343055442
-----------------------------------------------------------------------------------------------------------------
삼성전자 제품 제외
삼성전자 제품 제외
제품명: LG전자 그램17 스노우 화이트 노트북 17ZD90P-GX70K (i7-1165G7 43.1cm), 미포함, 256GB, 8GB
가격: 1,749,000원
평점: 5.0 (66)개
링크: https://www.coupang.com/vp/products/4618988569?itemId=5721255914&vendorItemId=73019994207
-----------------------------------------------------------------------------------------------------------------
삼성전자 제품 제외
삼성전자 제품 제외
로켓 배송만 포함합니다
삼성전자 제품 제외
삼성전자 제품 제외
삼성전자 제품 제외
제품명: Apple 2020년 맥북 에어 13, M1 8-Core, 16GB, SSD 256GB, 스페이스 그레이
가격: 1,480,180원
평점: 5.0 (815)개
링크: https://www.coupang.com/vp/products/4322481134?itemId=5504842876&vendorItemId=72804430326
-------------------------------------------------------------------------------------